## Imports

In [86]:
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib
import re


## Loading 

In [87]:
# ── Config ─────────────────────────────────────────────────────────────────────
INPUT_PATH  = '/Users/guill/Library/CloudStorage/OneDrive-NovaSBE/Cours Nova/' \
              'T3/Data Ecosystems and Governance/DG project/raw_credit_applications.json'
OUTPUT_PATH = os.path.join(os.path.dirname(INPUT_PATH), 'raw_credit_applications.csv')

# ── Load ───────────────────────────────────────────────────────────────────────
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"JSON file not found: {INPUT_PATH}")

with open(INPUT_PATH, 'r', encoding='utf-8') as f:
    try:
        data = json.load(f)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON: {e}")

if isinstance(data, dict):
    list_keys = [k for k, v in data.items() if isinstance(v, list)]
    data = data[list_keys[0]] if list_keys else [data]

## Export to CSV

In [88]:
# ── Build rows ─────────────────────────────────────────────────────────────────
rows = []
for record in data:

    # 1. applicant_info
    ai = record.get('applicant_info', {})
    full_name  = ai.get('full_name', '')
    name_parts = full_name.strip().split(None, 1)
    applicant_info = {
        'applicant_info_id'         : record.get('_id'),
        'applicant_info_full_name'  : full_name,
        'applicant_info_first_name' : name_parts[0] if len(name_parts) > 0 else '',
        'applicant_info_last_name'  : name_parts[1] if len(name_parts) > 1 else '',
        'applicant_info_email'      : ai.get('email'),
        'applicant_info_ssn'        : ai.get('ssn'),
        'applicant_info_ip_address' : ai.get('ip_address'),
        'applicant_info_gender'     : ai.get('gender'),
        'applicant_info_dob'        : ai.get('date_of_birth'),
        'applicant_info_zip_code'   : ai.get('zip_code'),
    }

    # 2. financials
    fin = record.get('financials', {})
    financials = {
        'financials_annual_income'         : fin.get('annual_income'),
        'financials_credit_history_months' : fin.get('credit_history_months'),
        'financials_debt_to_income'        : fin.get('debt_to_income'),
        'financials_savings_balance'       : fin.get('savings_balance'),
    }

    # 3. spending_behavior — one column per category
    spending_raw = record.get('spending_behavior', [])
    spending = {
        f'spending_behavior_{s["category"].lower().replace(" ", "_")}': s['amount']
        for s in spending_raw
    }

    # 4. decision
    dec = record.get('decision', {})
    decision = {
        'decision_loan_approved'    : dec.get('loan_approved'),
        'decision_rejection_reason' : dec.get('rejection_reason'),
        'decision_interest_rate'    : dec.get('interest_rate'),
        'decision_approved_amount'  : dec.get('approved_amount'),
        'decision_loan_purpose'     : record.get('loan_purpose'),
        'decision_notes'            : record.get('notes'),
    }

    # 5. processing_timestamp
    timestamp = {
        'processing_timestamp': record.get('processing_timestamp'),
    }

    rows.append({**applicant_info, **financials, **spending, **decision, **timestamp})

# ── Build DataFrame & enforce column order ─────────────────────────────────────
df = pd.DataFrame(rows)

spending_cols = sorted([c for c in df.columns if c.startswith('spending_behavior_')])
other_cols    = [c for c in df.columns if not c.startswith('spending_behavior_')]

def cols_starting(prefix):
    return [c for c in other_cols if c.startswith(prefix)]

ordered_cols = (
    cols_starting('applicant_info_')
    + cols_starting('financials_')
    + spending_cols
    + cols_starting('decision_')
    + cols_starting('processing_timestamp')
)

df = df[ordered_cols]

# ── Export ─────────────────────────────────────────────────────────────────────
df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')  # utf-8-sig for Excel compatibility
print(f"✓ Saved {len(df)} rows × {len(df.columns)} columns to {OUTPUT_PATH}")

✓ Saved 502 rows × 36 columns to /Users/guill/Library/CloudStorage/OneDrive-NovaSBE/Cours Nova/T3/Data Ecosystems and Governance/DG project/raw_credit_applications.csv


## Understanding the dataset

In [89]:
df = pd.read_csv("/Users/guill/Library/CloudStorage/OneDrive-NovaSBE/Cours Nova/T3/Data Ecosystems and Governance/DG project/raw_credit_applications.csv")

print("Shape:", df.shape)
display(df.head())

Shape: (502, 36)


,applicant_info_id,applicant_info_full_name,applicant_info_first_name,applicant_info_last_name,applicant_info_email,applicant_info_ssn,applicant_info_ip_address,applicant_info_gender,applicant_info_dob,applicant_info_zip_code,...,spending_behavior_transportation,spending_behavior_travel,spending_behavior_utilities,decision_loan_approved,decision_rejection_reason,decision_interest_rate,decision_approved_amount,decision_loan_purpose,decision_notes,processing_timestamp
0,app_200,Jerry Smith,Jerry,Smith,jerry.smith17@hotmail.com,596-64-4340,192.168.48.155,Male,2001-03-09,10036.0,...,NaN,NaN,NaN,False,algorithm_risk_score,NaN,NaN,NaN,NaN,2024-01-15T00:00:00Z
1,app_037,Brandon Walker,Brandon,Walker,brandon.walker2@yahoo.com,425-69-4784,10.1.102.112,M,1992-03-31,10032.0,...,NaN,NaN,NaN,False,algorithm_risk_score,NaN,NaN,NaN,NaN,NaN
2,app_215,Scott Moore,Scott,Moore,scott.moore94@mail.com,370-78-5178,10.240.193.250,Male,1989-10-24,10075.0,...,NaN,NaN,NaN,True,NaN,3.7,59000.0,vacation,NaN,NaN
3,app_024,Thomas Lee,Thomas,Lee,thomas.lee6@protonmail.com,194-35-1833,192.168.175.67,Male,1983-04-25,10077.0,...,NaN,NaN,NaN,True,NaN,4.3,34000.0,NaN,NaN,NaN
4,app_184,Brian Rodriguez,Brian,Rodriguez,brian.rodriguez86@aol.com,480-41-2475,172.29.125.105,M,1999-05-21,10080.0,...,NaN,NaN,NaN,False,algorithm_risk_score,NaN,NaN,NaN,NaN,2024-01-15T00:00:00Z


In [90]:
print("\nDataFrame Information:")
df.info()


DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 36 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   applicant_info_id                      502 non-null    object 
 1   applicant_info_full_name               502 non-null    object 
 2   applicant_info_first_name              502 non-null    object 
 3   applicant_info_last_name               502 non-null    object 
 4   applicant_info_email                   495 non-null    object 
 5   applicant_info_ssn                     497 non-null    object 
 6   applicant_info_ip_address              497 non-null    object 
 7   applicant_info_gender                  499 non-null    object 
 8   applicant_info_dob                     497 non-null    object 
 9   applicant_info_zip_code                500 non-null    float64
 10  financials_annual_income               497 non-nul

In [91]:
print("\nDescriptive Statistics for Numerical Columns Before Cleaning:")
print(df.describe())

print("\nDescriptive Statistics for Non-Numerical Columns Before Cleaning:")
print(df.describe(include='object'))


Descriptive Statistics for Numerical Columns Before Cleaning:
       applicant_info_zip_code  financials_annual_income  \
count               500.000000                497.000000   
mean              47660.026000              82705.096155   
std               39517.972093              28101.977862   
min               10001.000000                  0.000000   
25%               10048.000000              63000.000000   
50%               10097.500000              81000.000000   
75%               90244.000000             101000.000000   
max               90299.000000             171000.000000   

       financials_credit_history_months  financials_debt_to_income  \
count                        502.000000                 502.000000   
mean                          50.402390                   0.246195   
std                           31.234824                   0.136296   
min                          -10.000000                   0.050000   
25%                           27.250000       

In [92]:
# Types of variables

numerical_variables = df.select_dtypes(include=[np.number]).columns.tolist()
print("\nNumerical Variables:", numerical_variables)

categorical_variables = df.select_dtypes(include=['object']).columns.tolist()
print("\nCategorical Variables:", categorical_variables)


Numerical Variables: ['applicant_info_zip_code', 'financials_annual_income', 'financials_credit_history_months', 'financials_debt_to_income', 'financials_savings_balance', 'spending_behavior_adult_entertainment', 'spending_behavior_alcohol', 'spending_behavior_dining', 'spending_behavior_education', 'spending_behavior_entertainment', 'spending_behavior_fitness', 'spending_behavior_gambling', 'spending_behavior_groceries', 'spending_behavior_healthcare', 'spending_behavior_insurance', 'spending_behavior_rent', 'spending_behavior_shopping', 'spending_behavior_transportation', 'spending_behavior_travel', 'spending_behavior_utilities', 'decision_interest_rate', 'decision_approved_amount']

Categorical Variables: ['applicant_info_id', 'applicant_info_full_name', 'applicant_info_first_name', 'applicant_info_last_name', 'applicant_info_email', 'applicant_info_ssn', 'applicant_info_ip_address', 'applicant_info_gender', 'applicant_info_dob', 'decision_rejection_reason', 'decision_loan_purpose'

In [93]:
df_work = df.copy()

print((df_work == -1).sum().sort_values(ascending=False))

applicant_info_id                        0
applicant_info_full_name                 0
spending_behavior_gambling               0
spending_behavior_groceries              0
spending_behavior_healthcare             0
spending_behavior_insurance              0
spending_behavior_rent                   0
spending_behavior_shopping               0
spending_behavior_transportation         0
spending_behavior_travel                 0
spending_behavior_utilities              0
decision_loan_approved                   0
decision_rejection_reason                0
decision_interest_rate                   0
decision_approved_amount                 0
decision_loan_purpose                    0
decision_notes                           0
spending_behavior_fitness                0
spending_behavior_entertainment          0
spending_behavior_education              0
applicant_info_dob                       0
applicant_info_first_name                0
applicant_info_last_name                 0
applicant_i

In [94]:
# Values equal to -1 in numerical columns and Ages registered superior to 120 are likely placeholders for missing data.
# We will replace them with NaN to facilitate proper handling of missing values during analysis and modeling.

df_work.replace(-1, np.nan, inplace=True)

missing_table = pd.DataFrame({
    "missing_count": df_work.isna().sum(),
    "missing_%": df_work.isna().mean() * 100
})

# Round percentage
missing_table["missing_%"] = missing_table["missing_%"].round(2)

# Sort by highest missing %
missing_table = missing_table.sort_values(by="missing_count", ascending=False)

missing_table

,missing_count,missing_%
decision_notes,500,99.60
spending_behavior_adult_entertainment,497,99.00
spending_behavior_gambling,495,98.61
spending_behavior_alcohol,491,97.81
decision_loan_purpose,452,90.04
spending_behavior_shopping,448,89.24
spending_behavior_rent,443,88.25
spending_behavior_transportation,441,87.85
processing_timestamp,440,87.65
spending_behavior_education,438,87.25


## Cleaning

In [95]:
# ── Config ─────────────────────────────────────────────────────────────────────
INPUT_PATH  = '/Users/guill/Library/CloudStorage/OneDrive-NovaSBE/Cours Nova/' \
              'T3/Data Ecosystems and Governance/DG project/raw_credit_applications.csv'
OUTPUT_PATH = os.path.join(os.path.dirname(INPUT_PATH), 'cleaned_credit_applications.csv')

# ── Load ───────────────────────────────────────────────────────────────────────
df_work = pd.read_csv(INPUT_PATH, dtype={'applicant_info_zip_code': str})
print(f"Loaded {len(df_work)} rows × {len(df_work.columns)} columns")

issues_log = []  # track every change made

# ─────────────────────────────────────────────────────────────────────────────
# 1. DUPLICATES
# ─────────────────────────────────────────────────────────────────────────────

# Flag rows explicitly marked as duplicate in notes
duplicate_notes = df_work['decision_notes'].str.upper().str.contains('DUPLICATE', na=False)
df_work = df_work[~duplicate_notes]
issues_log.append(f"[Duplicates] Removed {duplicate_notes.sum()} rows flagged as DUPLICATE_ENTRY_ERROR in notes")

# Remove duplicate application IDs (keep first occurrence)
dup_ids = df_work['applicant_info_id'].duplicated(keep='first')
issues_log.append(f"[Duplicates] Removed {dup_ids.sum()} rows with duplicate applicant_info_id")
df_work = df_work[~dup_ids]

# Remove duplicate SSNs (keep first occurrence)
dup_ssns = df_work['applicant_info_ssn'].notna() & df_work['applicant_info_ssn'].duplicated(keep='first')
issues_log.append(f"[Duplicates] Removed {dup_ssns.sum()} rows with duplicate SSN")
df_work = df_work[~dup_ssns]

df_work = df_work.reset_index(drop=True)

# ─────────────────────────────────────────────────────────────────────────────
# 2. GENDER — standardize to Male / Female
# ─────────────────────────────────────────────────────────────────────────────
gender_map = {'M': 'Male', 'F': 'Female'}
before = df_work['applicant_info_gender'].value_counts(dropna=False).to_dict()
df_work['applicant_info_gender'] = df_work['applicant_info_gender'].replace(gender_map)
after = df_work['applicant_info_gender'].value_counts(dropna=False).to_dict()
issues_log.append(f"[Gender] Standardized abbreviations — before: {before} | after: {after}")

# ─────────────────────────────────────────────────────────────────────────────
# 3. EMAIL — null out malformed addresses
# ─────────────────────────────────────────────────────────────────────────────
email_pattern = r'^[\w\.-]+@[\w\.-]+\.\w{2,}$'
invalid_mask = df_work['applicant_info_email'].notna() & \
              ~df_work['applicant_info_email'].str.match(email_pattern, na=False)
invalid_emails = df_work.loc[invalid_mask, 'applicant_info_email'].tolist()
df_work.loc[invalid_mask, 'applicant_info_email'] = np.nan
issues_log.append(f"[Email] Nulled {len(invalid_emails)} invalid addresses: {invalid_emails}")

# ─────────────────────────────────────────────────────────────────────────────
# 4. PII — hash SSN and IP address (SHA-256, one-way)
# ─────────────────────────────────────────────────────────────────────────────
def hash_pii(value):
    if pd.isna(value):
        return np.nan
    return hashlib.sha256(str(value).encode()).hexdigest()

df_work['applicant_info_ssn']        = df_work['applicant_info_ssn'].apply(hash_pii)
df_work['applicant_info_ip_address'] = df_work['applicant_info_ip_address'].apply(hash_pii)
issues_log.append("[PII] SHA-256 hashed applicant_info_ssn and applicant_info_ip_address")

# ─────────────────────────────────────────────────────────────────────────────
# 5. ZIP CODE — clean and zero-pad to 5 digits
# ─────────────────────────────────────────────────────────────────────────────
df_work['applicant_info_zip_code'] = (
    df_work['applicant_info_zip_code']
    .str.replace(r'\.0$', '', regex=True)   # remove trailing .0 if read as float
    .str.strip()
    .str.zfill(5)                            # zero-pad to 5 digits
)
df_work['applicant_info_zip_code'] = df_work['applicant_info_zip_code'].replace('00nan', np.nan)
issues_log.append("[Zip Code] Converted to zero-padded 5-digit string")

# ─────────────────────────────────────────────────────────────────────────────
# 6. DATE OF BIRTH — parse and validate (must be a real date, age 18–100)
# ─────────────────────────────────────────────────────────────────────────────
# Normalize separators: replace '/' with '-' so all dates follow YYYY-MM-DD
df_work['applicant_info_dob'] = df_work['applicant_info_dob'].str.replace('/', '-', regex=False)

df_work['applicant_info_dob'] = pd.to_datetime(df_work['applicant_info_dob'], errors='coerce')

today = pd.Timestamp.today()
age = (today - df_work['applicant_info_dob']).dt.days / 365.25
invalid_age_mask = df_work['applicant_info_dob'].notna() & ((age < 18) | (age > 100))
issues_log.append(f"[DOB] Nulled {invalid_age_mask.sum()} rows with unrealistic age (outside 18–100)")
df_work.loc[invalid_age_mask, 'applicant_info_dob'] = np.nan
df_work['applicant_info_dob'] = df_work['applicant_info_dob'].dt.strftime('%Y-%m-%d')

# ─────────────────────────────────────────────────────────────────────────────
# 7. ANNUAL INCOME — flag zero income as suspicious
# ─────────────────────────────────────────────────────────────────────────────
# Round to nearest integer (no cents on an annual income)
df_work['financials_annual_income'] = df_work['financials_annual_income'].round(0).astype('Int64')

zero_income = df_work['financials_annual_income'] == 0
issues_log.append(f"[Income] Flagged {zero_income.sum()} rows with annual_income = 0")
df_work['financials_annual_income_flagged'] = zero_income
df_work.loc[zero_income, 'financials_annual_income'] = np.nan

# ─────────────────────────────────────────────────────────────────────────────
# 8. CREDIT HISTORY MONTHS — null out negative values
# ─────────────────────────────────────────────────────────────────────────────
negative_history_mask = df_work['financials_credit_history_months'] < 0
issues_log.append(f"[Credit History] Nulled {negative_history_mask.sum()} rows with negative credit_history_months: "
                  f"{df_work.loc[negative_history_mask, 'financials_credit_history_months'].tolist()}")
df_work.loc[negative_history_mask, 'financials_credit_history_months'] = pd.NA

# Cast to nullable integer to support NaN alongside integers
df_work['financials_credit_history_months'] = df_work['financials_credit_history_months'].astype('Int64')

# ─────────────────────────────────────────────────────────────────────────────
# 9. DEBT-TO-INCOME — flag and null values above 1.0 (economically impossible)
# ─────────────────────────────────────────────────────────────────────────────
dti_mask = df_work['financials_debt_to_income'] > 1.0
issues_log.append(f"[DTI] Nulled {dti_mask.sum()} rows with debt_to_income > 1.0: "
                  f"{df_work.loc[dti_mask, 'financials_debt_to_income'].tolist()}")
df_work.loc[dti_mask, 'financials_debt_to_income'] = np.nan

# ─────────────────────────────────────────────────────────────────────────────
# 10. SAVINGS BALANCE — null negatives, flag statistical outliers
# ─────────────────────────────────────────────────────────────────────────────

# Null negative values
negative_savings_mask = df_work['financials_savings_balance'] < 0
issues_log.append(f"[Savings] Nulled {negative_savings_mask.sum()} rows with negative savings_balance: "
                  f"{df_work.loc[negative_savings_mask, 'financials_savings_balance'].tolist()}")
df_work.loc[negative_savings_mask, 'financials_savings_balance'] = pd.NA

# Flag statistical outliers using IQR method (3x IQR threshold)
Q1 = df_work['financials_savings_balance'].quantile(0.25)
Q3 = df_work['financials_savings_balance'].quantile(0.75)
IQR = Q3 - Q1
outlier_mask = df_work['financials_savings_balance'] > Q3 + 3 * IQR
issues_log.append(f"[Savings] Flagged {outlier_mask.sum()} rows as savings_balance outliers "
                  f"(threshold: > {Q3 + 3 * IQR:,.0f}): "
                  f"{df_work.loc[outlier_mask, 'financials_savings_balance'].tolist()}")
df_work['financials_savings_balance_flagged'] = outlier_mask

# Cast to nullable integer
df_work['financials_savings_balance'] = df_work['financials_savings_balance'].astype('Int64')

# ─────────────────────────────────────────────────────────────────────────────
# 11. SPENDING BEHAVIOR — fill NaN with 0 (no spend in that category)
# ─────────────────────────────────────────────────────────────────────────────
spending_cols = [c for c in df_work.columns if c.startswith('spending_behavior_')]
df_work[spending_cols] = df_work[spending_cols].fillna(0).astype(int)
issues_log.append(f"[Spending] Filled NaN with 0 across {len(spending_cols)} spending columns")

# ─────────────────────────────────────────────────────────────────────────────
# 12. PROCESSING TIMESTAMP — parse, flag future dates
# ─────────────────────────────────────────────────────────────────────────────
df_work['processing_timestamp'] = pd.to_datetime(df_work['processing_timestamp'], errors='coerce', utc=True)
future_mask = df_work['processing_timestamp'].notna() & (df_work['processing_timestamp'] > pd.Timestamp.now(tz='UTC'))
issues_log.append(f"[Timestamp] Flagged {future_mask.sum()} rows with future processing_timestamp")
df_work['processing_timestamp_flagged'] = future_mask
df_work['processing_timestamp'] = df_work['processing_timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')


# ─────────────────────────────────────────────────────────────────────────────
# 13. DECISION — fill NaNs with meaningful values based on loan outcome
# ─────────────────────────────────────────────────────────────────────────────

# rejection_reason is N/A when loan was approved
df_work['decision_rejection_reason'] = df_work['decision_rejection_reason'].where(
    df_work['decision_rejection_reason'].notna(),
    other=df_work['decision_loan_approved'].map({True: 'N/A - Approved', False: 'Unknown'})
)

# interest_rate and approved_amount are N/A when loan was rejected
df_work['decision_interest_rate'] = df_work['decision_interest_rate'].where(
    df_work['decision_interest_rate'].notna(),
    other=df_work['decision_loan_approved'].map({True: 'Unknown', False: 'N/A - Rejected'})
)
df_work['decision_approved_amount'] = df_work['decision_approved_amount'].where(
    df_work['decision_approved_amount'].notna(),
    other=df_work['decision_loan_approved'].map({True: 'Unknown', False: 'N/A - Rejected'})
)

# loan_purpose — genuinely optional, fill with 'Not Specified'
df_work['decision_loan_purpose'] = df_work['decision_loan_purpose'].fillna('Not Specified')

issues_log.append("[Decision] Filled NaNs with context-aware labels based on loan_approved status")

# ─────────────────────────────────────────────────────────────────────────────
# 14. DROP low-value columns
# ─────────────────────────────────────────────────────────────────────────────
df = df_work.drop(columns=['decision_notes'])   # 500/502 were null, remaining 2 already actioned above
issues_log.append("[Cleanup] Dropped decision_notes column (99.6% null, content already actioned)")

# ─────────────────────────────────────────────────────────────────────────────
# 15. ENFORCE SCHEMA TYPES
# ─────────────────────────────────────────────────────────────────────────────

# --- Strings (all applicant_info fields should be str, not float) -------------
string_cols = [
    'applicant_info_id', 'applicant_info_full_name', 'applicant_info_first_name',
    'applicant_info_last_name', 'applicant_info_email', 'applicant_info_ssn',
    'applicant_info_ip_address', 'applicant_info_gender', 'applicant_info_dob',
    'applicant_info_zip_code',                    # was float64 → must be string
    'decision_rejection_reason', 'decision_loan_purpose'
]
for col in string_cols:
    df_work[col] = df_work[col].astype(str).replace({'nan': np.nan, 'None': np.nan})

# --- Integer (schema defines these as Integer) --------------------------------
int_cols = [
    'financials_credit_history_months',           # Integer per schema
    'financials_savings_balance',                 # Number but whole units
]
for col in int_cols:
    df_work[col] = pd.to_numeric(df_work[col], errors='coerce').astype('Int64')  # nullable int

# --- Float (Number in schema) -------------------------------------------------
float_cols = [
    'financials_annual_income',
    'financials_debt_to_income',
    'decision_interest_rate',
    'decision_approved_amount',
]
for col in float_cols:
    df_work[col] = pd.to_numeric(df_work[col], errors='coerce').astype(float)

# --- Boolean ------------------------------------------------------------------
df_work['decision_loan_approved'] = df_work['decision_loan_approved'].astype(bool)

issues_log.append("[Schema] Enforced correct dtypes across all columns per schema definition")

Loaded 502 rows × 36 columns


## Export the clean Dataset

In [96]:
# ─────────────────────────────────────────────────────────────────────────────
# EXPORT
# ─────────────────────────────────────────────────────────────────────────────
df_work.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')

print(f"\n✓ Cleaned dataset saved: {len(df_work)} rows × {len(df_work.columns)} columns")
print(f"  → {OUTPUT_PATH}")
print("\n─── Issues Log ───────────────────────────────────────────────────────")
for entry in issues_log:
    print(" •", entry)


✓ Cleaned dataset saved: 498 rows × 39 columns
  → /Users/guill/Library/CloudStorage/OneDrive-NovaSBE/Cours Nova/T3/Data Ecosystems and Governance/DG project/cleaned_credit_applications.csv

─── Issues Log ───────────────────────────────────────────────────────
 • [Duplicates] Removed 1 rows flagged as DUPLICATE_ENTRY_ERROR in notes
 • [Duplicates] Removed 1 rows with duplicate applicant_info_id
 • [Duplicates] Removed 2 rows with duplicate SSN
 • [Gender] Standardized abbreviations — before: {'Male': 193, 'Female': 193, 'F': 58, 'M': 52, nan: 2} | after: {'Female': 251, 'Male': 245, nan: 2}
 • [Email] Nulled 4 invalid addresses: ['mike johnson@gmail.com', 'test.user.outlook.com', 'john.doe@invalid', 'sarah.smith@']
 • [PII] SHA-256 hashed applicant_info_ssn and applicant_info_ip_address
 • [Zip Code] Converted to zero-padded 5-digit string
 • [DOB] Nulled 0 rows with unrealistic age (outside 18–100)
 • [Income] Flagged 1 rows with annual_income = 0
 • [Credit History] Nulled 2 rows w

In [97]:
print("\nDescriptive Statistics for Numerical Columns after cleaning:")
print(df_work.describe())

print("\nDescriptive Statistics for Non-Numerical Columns after cleaning:")
print(df_work.describe(include='object'))


Descriptive Statistics for Numerical Columns after cleaning:
       financials_annual_income  financials_credit_history_months  \
count                492.000000                             496.0   
mean               82752.912602                         50.612903   
std                27910.968499                         31.188889   
min                22000.000000                               0.0   
25%                63000.000000                             27.75   
50%                81000.000000                              48.5   
75%               101000.000000                              72.0   
max               171000.000000                             133.0   

       financials_debt_to_income  financials_savings_balance  \
count                 497.000000                       497.0   
mean                    0.241932                29516.022133   
std                     0.115793                16522.124074   
min                     0.050000                         0.0

In [98]:
# Types of variables

numerical_variables = df_work.select_dtypes(include=[np.number]).columns.tolist()
print("\nNumerical Variables:", numerical_variables)

categorical_variables = df_work.select_dtypes(include=['object']).columns.tolist()
print("\nCategorical Variables:", categorical_variables)


Numerical Variables: ['financials_annual_income', 'financials_credit_history_months', 'financials_debt_to_income', 'financials_savings_balance', 'spending_behavior_adult_entertainment', 'spending_behavior_alcohol', 'spending_behavior_dining', 'spending_behavior_education', 'spending_behavior_entertainment', 'spending_behavior_fitness', 'spending_behavior_gambling', 'spending_behavior_groceries', 'spending_behavior_healthcare', 'spending_behavior_insurance', 'spending_behavior_rent', 'spending_behavior_shopping', 'spending_behavior_transportation', 'spending_behavior_travel', 'spending_behavior_utilities', 'decision_interest_rate', 'decision_approved_amount']

Categorical Variables: ['applicant_info_id', 'applicant_info_full_name', 'applicant_info_first_name', 'applicant_info_last_name', 'applicant_info_email', 'applicant_info_ssn', 'applicant_info_ip_address', 'applicant_info_gender', 'applicant_info_dob', 'applicant_info_zip_code', 'decision_rejection_reason', 'decision_loan_purpose'